In [ ]:
"""Normalised processing and graphing"""

# Import libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
pd.set_option('display.max_rows', None)

# Apply theme
sns.set_theme(style="whitegrid")

In [ ]:
#############################################
# Fluorescence vs time normalised
#############################################
# Import csv file
input_file = "./221222 #55 1-3hrs-Batch_Analysis_23122022111831.csv"
df = pd.read_csv(input_file)

# Remove redundant columns
df = df.drop(columns=["Experiment Name", "Specimen Name", "$OP", "GUID", "Record Date", "All Events %Parent"
                      ,"All Events %Total","All Events FITC-A Mean","All Events PE-A Mean","All Events APC-A Mean"
                     ,"P1 %Parent","P1 %Total","P1 FITC-A Mean","P1 PE-A Mean","P1 APC-A Mean","P3 FITC-A Mean"
                     ,"P3 PE-A Mean","P3 APC-A Mean","P3 %Total","P2 %Parent", "P2 %Total"])

df2= df
strain = "W2"
stage = "Trophozoites"

dye_pop_strain_stage = [
    ("MitoSox","P2 PE-A Mean",strain, stage),
    ("CellRox","P2 APC-A Mean",strain, stage),
    ("Sytox","P3 %Parent", strain, stage),
    ("MitoTracker","P2 APC-A Mean", strain, stage),
    ("Fluo-4","P2 FITC-A Mean",strain, stage),
    ("JC-1","P2 PE-A Mean", strain, stage)
]

def time_vs_Mean_Fluorescence(dye,pop,strain,stage):
    
    time_list = ["1 hour", "2 hour", "3 hour"]
    for time in time_list:
        if dye == "Sytox":
            continue
        
        # Normalisaion processing
        ################################################
        # Calculate Mean
        df_Control = df2.loc[(df2.group == "Control")&(df2.time == time)&(df2.dye==dye)][pop]
        df_Control_mean = df_Control.mean()

        # Normalise values
        df2.loc[(df2.time == time)&(df2.dye==dye), pop
                   ]= (((df2[pop]/df_Control_mean))* 100) -100


    # Set parameters for plots
    data = df2.loc[(df2.dye == dye)]
    x="time"
    y=pop
    # Skip live/dead normalisation
    if dye == "Sytox":
        ylabel=f"Dead {strain} {stage} (% of all parasites)"
    else:
        ylabel=f"{dye} Fluorescence in {strain} {stage} (% change vs control)"
    
    xlabel="Time"
    height=6
    aspect=1.5
    errorbar="sd"
    hue="group"
    kind="point"
    capsize=0.05

    # Assign graph
    ax = sns.catplot(data=data, x=x, y=y, hue=hue,kind=kind,
                     height=height,aspect=aspect, errorbar=errorbar,capsize=capsize)
    ax.set(xlabel=xlabel, ylabel=ylabel)

    # Save png file and show plot
    plt.savefig(ylabel.replace(' ','_'))
    plt.show()

for dye,pop,strain,stage in dye_pop_strain_stage:
    time_vs_Mean_Fluorescence(dye,pop,strain,stage)

In [ ]:
########################################################
# % of Dead parasites vs Time
########################################################
time_list = ["1 hour","2 hour","3 hour"]
df_ld = df[['Well Name', 'Group', 'Time', 'P2 %Parent']]

for time in time_list:

    #Calculate Mean
    ################################################
    df_ld_Control = df_ld.loc[(df_ld.Group == "Control")&(df_ld.Time == time)]["P2 %Parent"]
    df_ld_Control_mean = df_ld_Control.mean()

    # Normalise values
    ################################################
    df_ld.loc[df_ld["Time"]==time, "P2 %Parent"
               ]= (((df_ld["P2 %Parent"]/df_ld_Control_mean))* 100) -100


x="Time"
y="P2 %Parent"
xlabel=""
ylabel="Dead W2 Rings (% change vs control)"
order = ['Control', 'TQ 2uM', 'MB 100nM', 'TQ&MB lo', 'TQ 10uM', 'MB 800nM', 'TQ&MB hi']
hue="Group"
kind = "point"
height=8
aspect=1.5

# Assign graph
plot = sns.catplot(data=df_ld,kind=kind,x=x, y=y, hue=hue ,height=height, aspect=aspect, capsize=0.05)
plot.set(xlabel=xlabel, ylabel=ylabel)

plt.savefig(ylabel.replace(' ', '_'))
plt.show()

In [ ]:
########################################################
# ATP vs time Normalised
########################################################

# Process data
################################################
### Import csv file
df = pd.read_csv("./ATP_E6.csv")
### isolate only iRBC
# df = df.loc[df.RBC == 'iRBC']
### drop RBC column (cant melt with 4 columns)
df = df.drop(columns=['Sheet1'])
### reshaped df to long form
dfm = df.melt('Group', var_name='Time', value_name='ATP')

# normalise data for each timepoint
################################################
time_list = ["1 Hour","2 Hour","3 Hour"]
df_time = dfm

for time in time_list:

    #Calculate Mean
    ################################################
    df_time_Control_ATP = df_time.loc[(df_time.Group == "Control")&(dfm.Time == time)]["ATP"]
    df_time_Control_ATP_mean = df_time_Control_ATP.mean()

    # Normalise values
    ################################################
    df_time.loc[df_time["Time"]==time, 'ATP'
               ]= (((df_time['ATP']/df_time_Control_ATP_mean))* 100) -100

x="Time"
y="ATP"
xlabel=""
ylabel="ATP Fluorescence in W2 Trophs (% change vs control)"
order = ['1 Hour', '2 Hour', '3 Hour']
hue="Group"
kind="point"

# Assign graph
plot = sns.catplot(data=df_time,kind=kind,x=x, y=y, hue=hue,height=8, aspect=1.5, capsize=0.05, order = order)
plot.set(xlabel=xlabel, ylabel=ylabel)

plt.savefig(ylabel.replace(' ', '_'))
plt.show()